# LCIA
A first strategy here is to map ecoinvent LCIA results for each electricity source to the classification used in ENTSO-E/energy-charts. Later on, we shall look at incorporating ENTSO-E/energy-charts data into ecoinvent, but this requires some aggregation first (it does not make sense to produce an "hourly ecoinvent").

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import sparse, io
from pathlib import Path
from pypardiso import spsolve
from scipy.linalg import block_diag

%load_ext jupyter_black


In [7]:
# To prevent a pypardiso crash later on
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [8]:
# Declaring our root folder
root = Path(".")

# ecoinvent folder
eifolder = (
    Path("..") / ".." / ".." / "ecoinvent" / "universal_matrix_export_3.10.1_cut-off"
)

## Import LCIA data

In [9]:
# import ecoinvent data
A_raw = pd.read_csv(eifolder / "A_public.csv", delimiter=";")
B_raw = pd.read_csv(eifolder / "B_public.csv", delimiter=";")
C_raw = pd.read_csv(eifolder / "C.csv", delimiter=";")

ee_index = pd.read_csv(eifolder / "ee_index.csv", delimiter=";")
ie_index = pd.read_csv(eifolder / "ie_index.csv", delimiter=";")
LCIA_index = pd.read_csv(eifolder / "LCIA_index.csv", delimiter=";")

In [10]:
# index ecoinvent data
A = sparse.csr_matrix(
    (A_raw["coefficient"], (A_raw["row"], A_raw["column"])),
    shape=(A_raw["row"].max() + 1, A_raw["column"].max() + 1),
)

B = sparse.csr_matrix(
    (B_raw["coefficient"], (B_raw["row"], B_raw["column"])),
    shape=(B_raw["row"].max() + 1, A_raw["column"].max() + 1),
)

C = sparse.csr_matrix(
    (C_raw["coefficient"], (C_raw["row"], C_raw["column"])),
    shape=(C_raw["row"].max() + 1, B_raw["row"].max() + 1),
)

In [11]:
# transform into dataframes
A_df = pd.DataFrame(
    A.toarray(),
    index=pd.MultiIndex.from_frame(ie_index),
    columns=pd.MultiIndex.from_frame(ie_index),
)

B_df = pd.DataFrame(
    B.toarray(),
    index=pd.MultiIndex.from_frame(ee_index),
    columns=pd.MultiIndex.from_frame(ie_index),
)

C_df = pd.DataFrame(
    C.toarray(),
    index=pd.MultiIndex.from_frame(LCIA_index),
    columns=pd.MultiIndex.from_frame(ee_index),
)

In [15]:
# Get precompiled lifecycle impacts
filename = root / "data" / "D_ei.pickle"

if filename.is_file():

    f = open(filename, "rb")
    D_df = pd.read_pickle(
        f
    )  # depending on the version of pandas this can be something else...

else:
    D_df = pd.read_excel(
        eifolder / ".." / "Cut-off Cumulative LCIA v3.10.1.xlsx",
        sheet_name="LCIA",
        index_col=[0, 1, 2, 3, 4, 5],
        header=[0, 1, 2, 3],
    )
    D_df.index.names = [
        "uuid",
        "activityName",
        "geography",
        "product",
        "unitName",
        "amount",
    ]
    D_df.columns.names = ["method", "category", "indicator", "unitName"]
    f = open(root / ".." / "data" / "D_ei.pickle", "wb")
    pickle.dump(D_df, f)
    f.close()

In [16]:
# Add UK as a region, we leave GB for backwards compatibility
D_df_GB = D_df.xs(
    "GB", level="geography", drop_level=False
)  # .rename(columns={'B':'C'}, level=0)
D_df_UK = D_df_GB.rename({"GB": "UK"}, level="geography")
D_df = pd.concat([D_df, D_df_UK])

In [18]:
def extract_lci(
    kw, unit="kg", excl_kws=["market"], regions=None, agg=True, regex=True, export=True
):

    if regions:
        index = ie_index[
            (ie_index["unitName"] == unit)
            & (ie_index["activityName"].str.contains(kw, regex=regex))
            & ~ie_index["activityName"].str.contains("|".join(excl_kws), regex=regex)
            & ie_index["geography"].str.contains("|".join(regions), regex=regex)
        ]
    else:
        index = ie_index[
            (ie_index["unitName"] == unit)
            & (ie_index["activityName"].str.contains(kw, regex=regex))
            & ~ie_index["activityName"].str.contains("|".join(excl_kws), regex=regex)
        ]

    cols = pd.MultiIndex.from_frame(index)
    df_A = -A_df.loc[:, cols]
    outputs = cols[(df_A.loc[cols].sum(1) == -1.0) & (df_A.loc[cols].sum() == -1.0)]
    df_A_clean = df_A[df_A.sum(1) != 0].drop(outputs)
    df_B = B_df.loc[:, cols]
    df_B_clean = df_B[df_B.sum(1) != 0]

    filename = root / "exports" / "{}.xlsx".format(kw + "_agg" * agg)

    if agg:
        df = pd.concat(
            [
                df_A_clean.groupby(["activityName", "product", "unitName"]).sum(),
                df_B_clean.groupby(["name", "compartment", "unitName"]).sum(),
            ],
            keys=["technosphere", "biosphere"],
            names=["source", "activityName", "product", "unitName"],
        )

    else:
        df = pd.concat(
            [
                df_A_clean.groupby(
                    ["activityName", "geography", "product", "unitName"]
                ).sum(),
                df_B_clean.groupby(
                    ["name", "compartment", "subcompartment", "unitName"]
                ).sum(),
            ],
            keys=["technosphere", "biosphere"],
            names=["source", "activityName", "geography", "product", "unitName"],
        )
    if export:
        df.style.set_properties(
            **{"font-size": "8pt", "font-family": "Arial Nova"}
        ).to_excel(filename)
        print("{} inventories saved as {}".format(kw, filename))

    return df

In [19]:
kw = "electricity, (low|medium|high) voltage"
unit = "kWh"
regions = None
excl_kws = ["#"]
regex = True

In [20]:
if Path("data/electricity_mixes.pickle").is_file():
    with open("data/electricity_mixes.pickle", "rb") as f:
        elec_mixes_ei = pd.read_pickle(f)
else:
    elec_mixes_ei = extract_lci(
        kw="electricity.*(low|medium|high) voltage",
        excl_kws=["#"],
        unit="kWh",
        agg=False,
        export=False,
    )

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\327906894.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (ie_index['activityName'].str.contains(kw, regex=regex)) &


In [22]:
elec_mixes_ei.xs("NL", level="geography", axis=1)[
    ("electricity, high voltage, production mix", "electricity, high voltage", "kWh")
]

index                                                                                                                                                          4978
source       activityName                                       geography                             product                                        unitName      
technosphere distribution network construction, electricity,... CA-QC                                 distribution network, electricity, low voltage km         0.0
             electricity production, coal, aluminium industry   CN                                    electricity, high voltage, aluminium industry  kWh        0.0
                                                                IAI Area, Africa                      electricity, high voltage, aluminium industry  kWh        0.0
                                                                IAI Area, Asia, without China and GCC electricity, high voltage, aluminium industry  kWh        0.0
                                                                IAI Area, EU27 & EFTA                 electricity, high voltage, aluminium industry  kWh        0.0
...                                                                                                                                                             ...
biosphere    Xylenes, unspecified                               air                                   unspecified                                    kg         0.0
                                                                water                                 unspecified                                    kg         0.0
             Zinc II                                            air                                   unspecified                                    kg         0.0
                                                                water                                 unspecified                                    kg         0.0
             t-Butyl methyl ether                               water                                 unspecified                                    kg         0.0

[3359 rows x 1 columns]

In [31]:
el_map = pd.read_excel(
    root / ".." / "data" / "electricity_sources.xlsx",
    sheet_name="electricity_sources",
    index_col=1,
).drop(["Unnamed: 0", "Unnamed: 46", "Unnamed: 47"], axis=1)
el_map.index.names = ["activityName"]

In [33]:
el_map.shape

(323, 44)

In [35]:
countries = list(
    pd.read_excel(
        root / ".." / "data" / "electricity_sources.xlsx",
        sheet_name="countries",
        index_col=None,
        header=None,
    )[0]
)

## Example, for one country
Question: since transmission only occurs at high voltage, should we include medium and low voltage?

### Working with the high voltage production mix only

In [37]:
extract = elec_mixes_ei["electricity, high voltage, production mix"]["NL"]
mix = extract.loc[extract.sum(1) != 0, :].iloc[:, 0]
mix

source        activityName                                                                             geography  product                    unitName
technosphere  electricity production, hard coal                                                        NL         electricity, high voltage  kWh         0.033692
              electricity production, hydro, run-of-river                                              NL         electricity, high voltage  kWh         0.000556
              electricity production, natural gas, combined cycle power plant                          NL         electricity, high voltage  kWh         0.296372
              electricity production, natural gas, conventional power plant                            NL         electricity, high voltage  kWh         0.149289
              electricity production, nuclear, pressure water reactor                                  NL         electricity, high voltage  kWh         0.046865
              electricity production, wi

In [38]:
el_map_c = el_map.reindex(mix.index, level=1).mul(mix, axis=0).loc["technosphere"]
el_map_c_norm = el_map_c / el_map_c.sum()

In [39]:
D_df_c = D_df.droplevel([0, 5]).reindex(el_map_c_norm.index)

It's a bit annoying not to have waste (presumably medium voltage) and solar (low voltage) in the mix. Does this production really get exchanged?

ENTSOE seems to assume that it does, but again, they probably account only for public generation? Private rooftop solar stays on the distribution grid, does it appear anywhere?

### Working with full (all voltages) mix

In [40]:
to_keep = [
    "market for electricity, low voltage",
    "electricity voltage transformation from medium to low voltage",
    "market for electricity, medium voltage",
    "electricity voltage transformation from high to medium voltage",
    "market for electricity, high voltage",
]

In [41]:
country = "NL"

extract = (
    elec_mixes_ei.loc["technosphere"][to_keep]
    .xs(country, level="geography", axis=1)
    .droplevel("index", axis=1)
)
mix = pd.concat(
    [extract.loc[extract.sum(1) != 0, :]], keys=[country], names=["geography"], axis=1
).reorder_levels(["geography", "activityName", "product", "unitName"])
mix.head()

geography                                                                                                                        NL  \
activityName                                                                                    market for electricity, low voltage   
product                                                                                                    electricity, low voltage   
unitName                                                                                                                        kWh   
geography activityName                                       product                   unitName                                       
NL        electricity production, hard coal                  electricity, high voltage kWh                                      0.0   
          electricity production, hydro, run-of-river        electricity, high voltage kWh                                      0.0   
          electricity production, natural gas, combined c... electricity, high voltage kWh                                      0.0   
          electricity production, natural gas, convention... electricity, high voltage kWh                                      0.0   
          electricity production, nuclear, pressure water... electricity, high voltage kWh                                      0.0   

geography                                                                                                                                                      \
activityName                                                                                    electricity voltage transformation from medium to low voltage   
product                                                                                                                              electricity, low voltage   
unitName                                                                                                                                                  kWh   
geography activityName                                       product                   unitName                                                                 
NL        electricity production, hard coal                  electricity, high voltage kWh                                                     0.0              
          electricity production, hydro, run-of-river        electricity, high voltage kWh                                                     0.0              
          electricity production, natural gas, combined c... electricity, high voltage kWh                                                     0.0              
          electricity production, natural gas, convention... electricity, high voltage kWh                                                     0.0              
          electricity production, nuclear, pressure water... electricity, high voltage kWh                                                     0.0              

geography                                                                                                                               \
activityName                                                                                    market for electricity, medium voltage   
product                                                                                                    electricity, medium voltage   
unitName                                                                                                                           kWh   
geography activityName                                       product                   unitName                                          
NL        electricity production, hard coal                  electricity, high voltage kWh                                         0.0   
          electricity production, hydro, run-of-river        electricity, high voltage kWh                                         0.0   
          electricity production, natural gas, combined c... electricity

In [43]:
mix_sq = mix.reindex(mix.index, axis=1).fillna(0)

In [44]:
# Need to put values to numpy first to fill the diagonal
diag_a = mix_sq.to_numpy()

In [45]:
np.fill_diagonal(diag_a, -1)

In [46]:
mix_sq.loc[:, :] = diag_a

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3596261849.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_sq.loc[:,:] = diag_a


In [47]:
L_sq = mix_sq.copy()
L_sq.values[:, :] = np.linalg.inv(-mix_sq)

In [49]:
mix_low_resolved = L_sq[
    ("NL", "market for electricity, low voltage", "electricity, low voltage", "kWh")
]
mix_low_resolved = mix_low_resolved.loc[
    mix_low_resolved.index.isin(el_map.index, level=1)
]

In [50]:
el_map_c = el_map.reindex(mix_low_resolved.index, level=1).mul(mix_low_resolved, axis=0)
el_map_c_norm = (el_map_c / el_map_c.sum()).swaplevel(0, 1)
el_map_c_norm.head()

,,,,Biomass,Brown coal,Fossil coal-derived gas,Fossil gas,Fossil oil,Fossil oil shale,Fossil peat,Geothermal,Hard coal,Hydro,...,Wind.1,Solar.1,FossilGasPower,FossilHardCoal,Nuclear.1,WastePower,BiomassPower,OtherPower,WKK Total,WindOffshoreC
activityName,geography,product,unitName,,,,,,,,,,,,,,,,,,,,,
"electricity production, hard coal",NL,"electricity, high voltage",kWh,0.0,NaN,0.0,0.000000,0.0,NaN,NaN,NaN,0.390518,NaN,...,0.0,0.0,0.000000,0.390518,0.0,NaN,0.0,NaN,0.0,0.0
"electricity production, hydro, run-of-river",NL,"electricity, high voltage",kWh,0.0,NaN,0.0,0.000000,0.0,NaN,NaN,NaN,0.000000,NaN,...,0.0,0.0,0.000000,0.000000,0.0,NaN,0.0,NaN,0.0,0.0
"electricity production, natural gas, combined cycle power plant",NL,"electricity, high voltage",kWh,0.0,NaN,0.0,0.467645,0.0,NaN,NaN,NaN,0.000000,NaN,...,0.0,0.0,0.467645,0.000000,0.0,NaN,0.0,NaN,0.0,0.0
"electricity production, natural gas, conventional power plant",NL,"electricity, high voltage",kWh,0.0,NaN,0.0,0.235563,0.0,NaN,NaN,NaN,0.000000,NaN,...,0.0,0.0,0.235563,0.000000,0.0,NaN,0.0,NaN,0.0,0.0
"electricity production, nuclear, pressure water reactor",NL,"electricity, high voltage",kWh,0.0,NaN,0.0,0.000000,0.0,NaN,NaN,NaN,0.000000,NaN,...,0.0,0.0,0.000000,0.000000,1.0,NaN,0.0,NaN,0.0,0.0


In [51]:
D_df_c = D_df.droplevel([0, 5]).reindex(el_map_c_norm.index)

In [53]:
el_map_c_norm.T.dot(D_df_c)

method                                                                   CML v4.8 2016 no LT  \
category                                                                 acidification no LT   
indicator                        acidification (incl. fate, average Europe total, A&B) no LT   
unitName                                                                           kg SO2-Eq   
Biomass                                                                    0.001266            
Brown coal                                                                      NaN            
Fossil coal-derived gas                                                    0.001953            
Fossil gas                                                                 0.000311            
Fossil oil                                                                 0.002151            
Fossil oil shale                                                                NaN            
Fossil peat                                                                     NaN            
Geothermal                                                                      NaN            
Hard coal                                                                  0.002119            
Hydro                                                                           NaN            
Hydro pumped storage consumption                                                NaN            
Hydro pumped storage generation                                                 NaN            
Hydro run-of-river                                                         0.000014            
Import balance                                                                  NaN            
Nuclear                                                                    0.000044            
Other renewables                                                                NaN            
Others                                                                          NaN            
Solar                                                                      0.000662            
Waste non-renewable                                                             NaN            
Waste renewable                                                                 NaN            
Wind                                                                       0.000106            
Fossil_Gas                                                                 0.000311            
Fossil_Hard_coal                                                           0.002119            
Fossil_Oil                                                                 0.002151            
Hydro_Pumped_Storage                                                            NaN            
Hydro_Run-of-river_and_poundage                                            0.000014            
Hydro_Water_Reservoir                                                           NaN            
Other_fossil                                                                    NaN            
Other_renewable                                                                 NaN            
Waste                                                                           NaN            
Wind_Onshore                                                               0.000108            
Fossil_Brown_coal/Lignite                                                       NaN            
Fossil_Coal-derived_gas                                                    0.001953            
Wind_Offshore                                                              0.000086            
Wind.1                                                                     0.000108            
Solar.1                                                                    0.000662            
FossilGasPower                                                             0.000311            
FossilHardCoal                                                  

## For all countries
TODO: same calculation with medium and low voltage production mixes (waste and solar)

In [54]:
countries_intersection = list(set(countries) & set(elec_mixes_ei.index.levels[2]))
n_countries = len(countries_intersection)

In [55]:
# Countries in ENTSOE but not in ecoinvent
# I think it's safe to use GB as a proxy for both
# the official name of the country is "the United Kingdom of Great Britain and Northern Ireland"...
set(countries) - set(elec_mixes_ei.index.levels[2])

{'NIE', 'UK'}

In [56]:
countries.append("GB")
countries.append("UK")

In [57]:
countries_intersection = sorted(
    list(set(countries) & set(elec_mixes_ei.index.levels[2]))
)
n_countries = len(countries_intersection)

In [58]:
extract = (
    elec_mixes_ei.loc["technosphere"][to_keep]
    .swaplevel(0, 1, axis=1)[countries_intersection]
    .droplevel(-1, axis=1)
)
extract.rename_axis(
    index={"geography": "geography_source"},
    columns={"geography": "geography_mix"},
    inplace=True,
)

In [59]:
level_order = ["geography_source", "activityName", "product", "unitName"]
mix = extract.loc[extract.sum(1) != 0, :].reorder_levels(level_order)

In [60]:
mix_sq = mix.reindex(mix.index, axis=1).fillna(0)
# np.fill_diagonal(mix_sq.values, -1)
diag_a = mix_sq.to_numpy()
np.fill_diagonal(diag_a, -1)
mix_sq.loc[:, :] = diag_a

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3222477243.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_sq.loc[:,:] = diag_a


In [61]:
L_sq = mix_sq.copy()
L_sq.values[:, :] = np.linalg.inv(-mix_sq)

In [62]:
# We build the mix share matrix for low voltages, and transform it into a Series
mix_low_resolved = L_sq.loc[
    :,
    (
        slice(None),
        "market for electricity, low voltage",
        "electricity, low voltage",
        "kWh",
    ),
]
mix_low_resolved = (
    mix_low_resolved.loc[mix_low_resolved.index.isin(el_map.index, level=1)]
    .droplevel([1, 2, 3], axis=1)
    .sort_index()
)
mix_low_resolved.rename_axis(
    columns={"geography_source": "geography_mix"}, inplace=True
)

mix_low_col = mix_low_resolved.stack().sort_index()
mix_low_col = mix_low_col[mix_low_col != 0].sort_index()
# mix_low_col[mix_low_col==0] = 1e-12

In [63]:
to_exclude = [
    "electricity voltage transformation from high to medium voltage",
    "electricity voltage transformation from medium to low voltage",
    "electricity, high voltage, production mix",
    "market for electricity, high voltage",
    "market for electricity, low voltage",
    "market for electricity, medium voltage",
    "electricity voltage transformation, residual mix, from high to medium voltage",
    "electricity, low voltage, residual mix",
    "electricity, high voltage, residual mix",
    "electricity, medium voltage, residual mix",
    "electricity voltage transformation, residual mix, from medium to low voltage",
]

In [64]:
### There are inventories with emission factors out there, but they don't appear in any mix
available_LCIs = D_df.loc[
    (
        slice(None),
        slice(None),
        slice(None),
        [
            "electricity, high voltage",
            "electricity, medium voltage",
            "electricity, low voltage",
        ],
        slice(None),
        slice(None),
    ),
    :,
].droplevel(["uuid", "amount"])
available_LCIs.index.rename({"geography": "geography_source"}, inplace=True)
available_LCIs = available_LCIs.reorder_levels(level_order).sort_index()
available_LCIs = available_LCIs.drop(
    available_LCIs.loc[(slice(None), to_exclude, slice(None), slice(None)), :].index
)

imports = available_LCIs.index.get_level_values("activityName").str.startswith(
    "electricity, high voltage, import from "
)

available_LCIs = available_LCIs[~imports]

In [65]:
for country in mix_low_resolved.index.get_level_values("geography_source").unique():

    if country == "RoW":
        continue

    LCIs_to_add = set(available_LCIs.xs(country).index) - set(
        mix_low_resolved.xs(country)[country].index
    )

    for LCI in LCIs_to_add:
        mix_low_resolved.loc[(country, *LCI), country] = 1e-12
        if country == "GB":
            mix_low_resolved.loc[("UK", *LCI), "UK"] = 1e-12

    # mix_low_resolved.loc[]
    print(country, LCIs_to_add, "\n")

mix_low_resolved.fillna(0)

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Lo

AT {('electricity, from municipal waste incineration to generic market for electricity, medium voltage', 'electricity, medium voltage', 'kWh'), ('treatment of coal gas, in power plant', 'electricity, high voltage', 'kWh'), ('treatment of blast furnace gas, in power plant', 'electricity, high voltage', 'kWh'), ('electricity production, photovoltaic, 570kWp open ground installation, multi-Si', 'electricity, low voltage', 'kWh')} 

BA {('treatment of coal gas, in power plant', 'electricity, high voltage', 'kWh'), ('treatment of blast furnace gas, in power plant', 'electricity, high voltage', 'kWh'), ('heat and power co-generation, natural gas, conventional power plant, 100MW electrical', 'electricity, high voltage', 'kWh')} 

BE {('electricity, from municipal waste incineration to generic market for electricity, medium voltage', 'electricity, medium voltage', 'kWh'), ('electricity production, hard coal', 'electricity, high voltage', 'kWh')} 

BG {('electricity, from municipal waste incine

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Lo

CH {('treatment of waste mineral oil, hazardous waste incineration, with energy recovery', 'electricity, medium voltage', 'kWh'), ('electricity production, hydro, pumped storage', 'electricity, high voltage', 'kWh'), ('wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', 'electricity, low voltage', 'kWh'), ('treatment of waste polypropylene, sanitary landfill', 'electricity, medium voltage', 'kWh'), ('electricity, high voltage, biofuels, import from Germany', 'electricity, high voltage', 'kWh'), ('heat and power co-generation, natural gas, 160kW electrical, lambda=1', 'electricity, low voltage', 'kWh'), ('treatment of waste wood, untreated, sanitary landfill', 'electricity, medium voltage', 'kWh'), ('treatment of waste newspaper, sanitary landfill', 'electricity, medium voltage', 'kWh'), ('heat and power co-generation, wood chips, 6667 kW', 'electricity, high voltage', 'kWh'), ('heat and power co-generation, natural gas, 50kW electrical, lean bur

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\3486186891.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  mix_low_resolved.loc[(country,*LCI), country] = 1e-12
C:\Users\Gibon\AppData\Lo

geography_mix                                                                                            AL  \
geography_source activityName                                       product                   unitName        
AT               electricity production, deep geothermal            electricity, high voltage kWh       0.0   
                 electricity production, hard coal                  electricity, high voltage kWh       0.0   
                 electricity production, hydro, pumped storage      electricity, high voltage kWh       0.0   
                 electricity production, hydro, reservoir, alpin... electricity, high voltage kWh       0.0   
                 electricity production, hydro, run-of-river        electricity, high voltage kWh       0.0   
...                                                                                                     ...   
UA               electricity production, hydro, pumped storage      electricity, high voltage kWh       0.0   
                 treatment of blast furnace gas, in power plant     electricity, high voltage kWh       0.0   
                 treatment of coal gas, in power plant              electricity, high voltage kWh       0.0   
                 heat and power co-generation, wood chips, 6667 kW  electricity, high voltage kWh       0.0   
                 electricity production, lignite                    electricity, high voltage kWh       0.0   

geography_mix                                                                                            AM  \
geography_source activityName                                       product                   unitName        
AT               electricity production, deep geothermal            electricity, high voltage kWh       0.0   
                 electricity production, hard coal                  electricity, high voltage kWh       0.0   
                 electricity production, hydro, pumped storage      electricity, high voltage kWh       0.0   
                 electricity production, hydro, reservoir, alpin... electricity, high voltage kWh       0.0   
                 electricity production, hydro, run-of-river        electricity, high voltage kWh       0.0   
...                                                                                                     ...   
UA               electricity production, hydro, pumped storage      electricity, high voltage kWh       0.0   
                 treatment of blast furnace gas, in power plant     electricity, high voltage kWh       0.0   
                 treatment of coal gas, in power plant              electricity, high voltage kWh       0.0   
                 heat and power co-generation, wood chips, 6667 kW  electricity, high voltage kWh       0.0   
                 electricity production, lignite                    electricity, high voltage kWh       0.0   

geography_mix                                                                                                     AT  \
geography_source activityName                                       product                   unitName                 
AT               electricity production, deep geothermal            electricity, high voltage kWh       7.807336e-07   
                 electricity production, hard coal                  electricity, high voltage kWh       2.307101e-03   
                 electricity production, hydro, pumped storage      electricity, high voltage kWh       3.904697e-02   
                 electricity production, hydro, reservoir, alpin... electricity, high voltage kWh       1.208470e-01   
                 electricity production, hydro, run-of-river        electricity, high voltage kWh       3.625046e-01   
...                                                                                                              ...   
UA               electricity production, hydro, pumped storage      electricity, high voltage kWh       0.000000e+00   
                 treatment of bl

In [66]:
mismatch_source_mix = mix_low_col[
    ~(mix_low_col.index.get_level_values(0) == mix_low_col.index.get_level_values(4))
]

In [67]:
# There are mismatches between source and mix, because of the "RoW" values
mismatch_source_mix.index.get_level_values(0).unique()

Index(['RoW'], dtype='object', name='geography_source')

In [68]:
map_country_techs = pd.DataFrame(
    block_diag(*[el_map.values for _ in range(n_countries)]),
    index=pd.MultiIndex.from_product(
        [countries_intersection, el_map.index],
        names=["geography_source", "activityName"],
    ),
    columns=pd.MultiIndex.from_product(
        [countries_intersection, el_map.columns], names=["geography_mix", "source"]
    ),
)

In [69]:
for row in mismatch_source_mix.items():

    old_region = row[0][4]
    new_region = row[0][0]
    activity = row[0][1]

    match = map_country_techs.loc[(old_region, activity)] == 1
    idx_match = map_country_techs.loc[(old_region, activity), match].index

    print(f"Fixing match for {old_region}, activity {activity}, moving to {new_region}")

    map_country_techs.loc[(old_region, activity), idx_match] = 0
    map_country_techs.loc[(new_region, activity), idx_match] = 1

Fixing match for HR, activity electricity production, deep geothermal, moving to RoW
Fixing match for HU, activity electricity production, hard coal, moving to RoW
Fixing match for PL, activity electricity production, hard coal, moving to RoW
Fixing match for AL, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for AM, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for AZ, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for BY, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for GE, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for MD, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for ME, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for XK, activity electricity production, hydro, run-of-river, moving to RoW
Fixing match for ME, activity electricity pro

In [70]:
map_country_techs.fillna(0, inplace=True)

In [71]:
el_map_all = map_country_techs.reindex(mix_low_resolved.index) * mix_low_resolved

In [72]:
el_map_all_norm = (el_map_all / el_map_all.sum()).fillna(0)

In [75]:
el_map_all_norm["NL"]["WKK Total"].nlargest(10)

geography_source  activityName                                                                             product                    unitName
NL                heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical  electricity, high voltage  kWh         0.321410
                  heat and power co-generation, natural gas, conventional power plant, 100MW electrical    electricity, high voltage  kWh         0.266226
                  heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014                 electricity, high voltage  kWh         0.220341
                  heat and power co-generation, hard coal                                                  electricity, high voltage  kWh         0.189942
                  heat and power co-generation, biogas, gas engine                                         electricity, high voltage  kWh         0.001265
                  heat and power co-generation, oil                               

In [76]:
el_map_all_norm["NL"]["Solar"].nlargest(10)

geography_source  activityName                                                                                     product                    unitName
NL                electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted   electricity, low voltage   kWh         0.546917
                  electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted  electricity, low voltage   kWh         0.439678
                  electricity production, photovoltaic, 570kWp open ground installation, multi-Si                  electricity, low voltage   kWh         0.013405
AT                electricity production, deep geothermal                                                          electricity, high voltage  kWh         0.000000
                  electricity production, hard coal                                                                electricity, high voltage  kWh         0.000000
                  electricity prod

In [83]:
el_map_all_norm["NL"]["WastePower"].nlargest(10)

geography_source  activityName                                                                                     product                    unitName
AT                electricity production, deep geothermal                                                          electricity, high voltage  kWh         0.0
                  electricity production, hard coal                                                                electricity, high voltage  kWh         0.0
                  electricity production, hydro, pumped storage                                                    electricity, high voltage  kWh         0.0
                  electricity production, hydro, reservoir, alpine region                                          electricity, high voltage  kWh         0.0
                  electricity production, hydro, run-of-river                                                      electricity, high voltage  kWh         0.0
                  electricity production, natural gas, comb

In [49]:
# el_map_all_norm = pd.concat([el_map_all_norm], keys=['electricity, high voltage'], names=['product']).reorder_levels([2,1,0])

In [84]:
D_filtered = (
    D_df.rename_axis(index={"geography": "geography_source"})
    .droplevel([0, 5])
    .swaplevel(0, 1)
    .reindex(el_map_all_norm.index)
)

In [85]:
ENTSOE_CFs = el_map_all_norm.T.dot(D_filtered)

In [86]:
ENTSOE_CFs_GB = ENTSOE_CFs.xs(
    "GB", level="geography_mix", drop_level=False
)  # .rename(columns={'B':'C'}, level=0)
ENTSOE_CFs_UK = ENTSOE_CFs.rename({"GB": "UK"}, level="geography_mix").xs(
    "UK", level="geography_mix", drop_level=False
)
ENTSOE_CFs = pd.concat([ENTSOE_CFs, ENTSOE_CFs_UK])

In [87]:
ENTSOE_CFs_valid = ENTSOE_CFs[ENTSOE_CFs.sum(1) != 0]

In [88]:
mix_other = D_df.xs(
    ("UCTE", "electricity, low voltage"), level=["geography", "product"]
)
mix_other.index = ["Mix_Other"]

In [89]:
### Complete potentially missing sources

source = "Wind_Offshore"
values = ENTSOE_CFs_valid.loc[(slice(None), source), :]
default_values = values.mean()[ENTSOE_CFs_valid.columns].values
missing_countries = set(countries_intersection) - set(
    values.index.get_level_values("geography_mix")
)
ENTSOE_CFs_valid.loc[("AL", source), :] = default_values
ENTSOE_CFs_valid.loc[("BG", source), :] = default_values
ENTSOE_CFs_valid.loc[("CY", source), :] = default_values
ENTSOE_CFs_valid.loc[("GR", source), :] = default_values
ENTSOE_CFs_valid.loc[("HR", source), :] = default_values
ENTSOE_CFs_valid.loc[("IT", source), :] = default_values
ENTSOE_CFs_valid.loc[("LT", source), :] = default_values
ENTSOE_CFs_valid.loc[("LV", source), :] = default_values
ENTSOE_CFs_valid.loc[("ME", source), :] = default_values
ENTSOE_CFs_valid.loc[("MT", source), :] = default_values
ENTSOE_CFs_valid.loc[("RO", source), :] = default_values
ENTSOE_CFs_valid.loc[("RU", source), :] = default_values
ENTSOE_CFs_valid.loc[("SI", source), :] = default_values
ENTSOE_CFs_valid.loc[("TR", source), :] = default_values
ENTSOE_CFs_valid.loc[("UA", source), :] = default_values

source = "Other_fossil"
values = ENTSOE_CFs_valid.loc[(slice(None), source), :]
default_values = values.mean()[ENTSOE_CFs_valid.columns].values
missing_countries = set(countries_intersection) - set(
    values.index.get_level_values("geography_mix")
)
for country in missing_countries:
    ENTSOE_CFs_valid.loc[(country, source), :] = default_values

# source = 'Other renewables'
# values = ENTSOE_CFs_valid.loc[(slice(None),source),:]
# default_values = values.mean()[ENTSOE_CFs_valid.columns].values
# missing_countries = set(countries_intersection) - set(values.index.get_level_values('geography_mix'))
# for country in missing_countries:
#     ENTSOE_CFs_valid.loc[(country,source),:] = default_values

source = "Waste"
for country in countries_intersection:
    ENTSOE_CFs_valid.loc[(country, source), :] = 0

source = "Hydro_Water_Reservoir"
values = ENTSOE_CFs_valid.loc[(slice(None), source), :]
default_values = values.mean()[ENTSOE_CFs_valid.columns].values
ENTSOE_CFs_valid.loc[("LU", source), :] = default_values

source = "Hydro_Water_Reservoir"
values = ENTSOE_CFs_valid.loc[(slice(None), source), :]
default_values = values.mean()[ENTSOE_CFs_valid.columns].values
ENTSOE_CFs_valid.loc[("LU", source), :] = default_values

C:\Users\Gibon\AppData\Local\Temp\ipykernel_30520\753843502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ENTSOE_CFs_valid.loc[('AL',source),:] = default_values


In [91]:
ENTSOE_CFs_valid.to_excel(root / ".." / "output" / "ENTSOE_CFs.xlsx")

OSError: Cannot save file into a non-existent directory: '..\output'

In [93]:
ENTSOE_CFs_valid.loc["NL"]

method                                                                  CML v4.8 2016 no LT  \
category                                                                acidification no LT   
indicator                       acidification (incl. fate, average Europe total, A&B) no LT   
unitName                                                                          kg SO2-Eq   
source                                                                                        
Biomass                                                                   0.001266            
Fossil coal-derived gas                                                   0.001953            
Fossil gas                                                                0.000311            
Fossil oil                                                                0.002151            
Hard coal                                                                 0.002119            
Hydro run-of-river                                                        0.000014            
Nuclear                                                                   0.000044            
Solar                                                                     0.000662            
Wind                                                                      0.000106            
Fossil_Gas                                                                0.000311            
Fossil_Hard_coal                                                          0.002119            
Fossil_Oil                                                                0.002151            
Hydro_Run-of-river_and_poundage                                           0.000014            
Wind_Onshore                                                              0.000108            
Fossil_Coal-derived_gas                                                   0.001953            
Wind_Offshore                                                             0.000086            
Wind.1                                                                    0.000108            
Solar.1                                                                   0.000662            
FossilGasPower                                                            0.000311            
FossilHardCoal                                                            0.002119            
Nuclear.1                                                                 0.000044            
BiomassPower                                                              0.001266            
WKK Total                                                                 0.000854            
WindOffshoreC                                                             0.000086            
Other_fossil                                                              0.003564            
Waste                                                                     0.000000            

method                                                                   \
category                                           climate change no LT   
indicator                       global warming potential (GWP100) no LT   
unitName                                                      kg CO2-Eq   
source                                                                    
Biomass                                                        0.054980   
Fossil coal-derived gas                                        0.592619   
Fossil gas                                                     0.539823   
Fossil oil                                                     1.156062   
Hard coal                                                      0.978428   
Hydro run-of-river                                             0.004342   
Nuclear                                                        0.007052   
Solar                                                          0.103743   
Wind                                                           0.017437   
Foss

In [57]:
# ENTSOE_CFs_for_ecodynelec = ENTSOE_CFs_valid.loc[(slice(None), to_keep),:].sort_index()
# ENTSOE_CFs_for_ecodynelec.index = ENTSOE_CFs_for_ecodynelec.index.swaplevel().to_frame().agg('_'.join, axis=1)

In [58]:
# ENTSOE_CFs_for_ecodynelec.loc['Mix_Other'] = mix_other.loc['Mix_Other']

In [59]:
# ENTSOE_CFs_for_ecodynelec.to_excel(root / 'output' / 'ENTSOE_CFs_for_ecodynelec.xlsx')

In [95]:
el_map_all_norm.to_csv("../data/el_map_all_norm_w_ned.csv")